# TEST

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from tools.network import Network
from tools.decode_raw import decode_img_seg, decode_img_seg_test

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train')

feature_dict = {
    'img': 'bytes', 
    'label': 'bytes',
    'height': 'int', 
    'width': 'int',
    'channels': 'int',
    'n_class': 'int'
}

network = Network()

In [ ]:
seg_train_gen_params = {
    'csv_path': csv_path,
    'train_path': train_path,
    'height': 256,
    'width': 1600, 
    'col': False,
    'sep':'[_,]',
    'n_class': 5
}

gen = network.seg_train_gen(**seg_train_gen_params)

write_tfr_params = {
    'data_generator': gen,
    'count': network.count(train_path),
    'tfrpath': tfr_path,
    'feature_dict': feature_dict,
    'shards': 10,
    'compression': 'GZIP', 
    'c_level': 1
}

network.write_tfr(**write_tfr_params)

In [3]:
rt_params = {
    'feature_dict': feature_dict, 
    'decode_raw': decode_img_seg,
    'tfr_path': os.path.join(tfr_path, '*.tfrecord'),
    'shuffle_buffer': 100,
    'compression': 'GZIP'
}

readtrain_params = {
    'rt_params': rt_params, 
    'train_path': train_path,
    'valid_percentage': 0.2,
    'epoch': 50,
    'batch_size': 4,
    'reshape': [128, 800],
    'reshape_method': 3}


network.readtrain(**readtrain_params)

W0925 02:52:24.694063 139983241951040 deprecation.py:323] From /home/chamhaolee/anaconda3/envs/severstal/lib/python3.7/site-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0925 02:52:24.722394 139983241951040 deprecation.py:323] From /home/chamhaolee/Severstal/tools/data_tfr.py:146: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [4]:
model_params = {
    'num_layers': 5,
    'feature_growth_rate': 16,
    'n_class': 5,
    'channels': 3,
    'padding': 'SAME',
    'dropout_rate':0.25
}

network.model(
    model_name='unet',
    model_params=model_params,
    loss='neg_dice', 
    metric='dice',
    optimizer='momentun',
    rate=1e-5)

W0925 02:52:26.159958 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/model_component.py:23: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0925 02:52:26.198492 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/model_component.py:62: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0925 02:52:26.739530 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/model_component.py:99: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.



In [5]:
network.train(ckpt_dir=os.path.join('..', 'tmp', 'ckpt'), early_stopping = 5, verbose = 2, retrain=False)

W0925 02:52:27.571702 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/network.py:80: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0925 02:52:27.643533 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/network.py:81: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0925 02:52:28.150006 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/network.py:91: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0925 02:52:28.153431 139983241951040 deprecation_wrapper.py:119] From /home/chamhaolee/Severstal/tools/network.py:91: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.



After 1 epoch, train neg_dice is 0.04540000110864639, valid dice is 0.9842000007629395
After 2 epoch, train neg_dice is 0.0348999984562397, valid dice is 0.9829000234603882


E0925 03:29:12.778359 139983241951040 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

In [ ]:
network.test()